In [3]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langserve import RemoteRunnable
from langchain_core.runnables import RunnableParallel
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
import os
from langchain_upstage import UpstageGroundednessCheck

In [4]:
from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [10]:
import os
from langchain_upstage import UpstageGroundednessCheck
 
groundedness_check = UpstageGroundednessCheck()
 
response = groundedness_check.invoke({
    "context": "애플코인은 2009년에 창시된 분산형 디지털 통화로, 블록체인 기술을 기반으로 한다. 2023년 7월 기준 애플코인의 가격은 약 3만 달러이다.",
    "answer": "애플코인의 가격은 2023년 7월 기준으로 5만 달러이다."
})
print(response)
response = groundedness_check.invoke({
    "context": "애플뱅크는 2017년에 설립된 대한민국의 인터넷 전문 은행이다. 비대면으로 계좌를 개설할 수 있으며, 다양한 금융 서비스를 제공한다. 2023년 기준 애플뱅크의 고객 수는 약 1,500만 명에 이른다.",
    "answer": "애플뱅크는 2023년 기준 고객 수가 1,500만 명에 이른다."
})
print(response)

notGrounded
grounded


In [11]:
groundedness_check.invoke({
    "context": """
## 본문: 라마 3 벤치마크 결과 (사진=메타)


라마 3는 객관식 문제(MMLU)와 코딩(HumanEval)에는 강하지만, 70B의 경우 수학 단어 문제(MATH) 해결이나 대학원생 수준의 객관식 문제(GPQA)에서는 제미나이 프로 1.5에 떨어졌다. 
특히 인간 선호도에서 경쟁 모델을 앞서는 것으로 알려졌다.
조언 요청, 브레인스토밍, 분류, 비공개 질문 답변, 코딩, 창의적인 글쓰기, 추출, 공개 질문 답변, 추론, 재작성 및 요약 등 12가지 주요 사용 사례를 포함한 1800개 프롬프트 구축 데이터셋에 대한 인간 평가에서 오픈AI의 'GPT-3.5', 미스트랄 7B, 클로드 3 소네트보다 높게 평가됐다.


라마 3 인간 평가 결과 (사진=메타)
### 출처: https://www.aitimes.com/news/articleView.html?idxno=158943

## 본문: 라마 3 인간 평가 결과 (사진=메타)


허깅페이스에 따르면, 라마 3는 공개 후 몇시간만에 LLM 리더보드 1위에 오르며 역대 가장 빠른 1위 달성 기록을 세웠다.
또 이전 라마 1과 2를 기반으로 3만개 이상의 새로운 모델이 출시됐으며, 라마 2 모델은 1700억번 다운로드됐다는 통계치도 공개해 눈길을 모았다. 
다만 라마 3는 완전한 오픈 소스가 아니다. 연구용 및 상업용으로 모두 사용할 수 있지만, 개발자가 다른 생성 모델을 훈련하기 위해 모델을 사용하는 것을 금지한다.


메타 AI (사진=메타)
### 출처: https://www.aitimes.com/news/articleView.html?idxno=158943

## 본문: 특히 15조개 이상의 토큰을 동원, 학습량이 라마 2 대비 7배 이상 많으며 코드량은 4배 더 많다. 다만 데이터셋은 공개하지 않았다.
이후 미세조정에는 일상적인 질문부터 과학·기술·공학·수학(STEM), 코딩, 역사 지식에 이르기까지 다양한 분야의 데이터셋이 사용됐다. 훈련 규모를 확대하는 것은 물론, 고도화된 ‘지시 미세조정(instruction fine-tuning)’ 과정도 진행했다. 
또 라마 3는 라마 2보다 2배 큰 8000토큰의 컨텍스트 길이를 지원한다.
오픈 소스라는 점을 감안, 안전하고 책임감 있는 개발과 사용을 위한 다양한 안전장치도 마련했다고 밝혔다. 전문가와 자동화된 도구를 활용한 레드팀 테스트를 통해 부적절한 답변의 가능성을 최소화했다고 전했다.
### 출처: https://www.aitimes.com/news/articleView.html?idxno=158943

## 본문: ii 【2023년도 중앙선거관리위원회 정책연구용역 보고서】 『생성형 AI 신기술 도입에 따른  선거 규제 연구』 연구책임자 :  김  주  희(국 립 부 경 대 학 교) 공동연구자 : 차  재  권(국 립 부 경 대 학 교) 김  현  정(동 아 대 학 교) 조  성  복(국 민 대 학 교) 연구보조원 :  박  서  현(국 립 부 경 대 학 교) 권  수  민(국 립 부 경 대 학 교) 본 연구보고서는 2023년도 중앙선거관리위원회 정책연구용역 과제로서 연구내용은 중앙 선거관리위원회의 공식 견해가 아님.
### 출처: /Users/user/kykdev/7000_AI/test_langchain/assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf

## 본문: 연구책임자 :  김  주  희(국 립 부 경 대 학 교) 공동연구자 : 차  재  권(국 립 부 경 대 학 교) 김  현  정(동 아 대 학 교) 조  성  복(국 민 대 학 교) 연구보조원 :  박  서  현(국 립 부 경 대 학 교) 권  수  민(국 립 부 경 대 학 교) 국립부경대학교 산학협력단 2023년도 중앙선거관리위원회 정책연구용역 보고서 생성형 AI 신기술 도입에 따른  선거 규제 연구
### 출처: /Users/user/kykdev/7000_AI/test_langchain/assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf

## 본문: 2 - 2014년 ~ 현재의 생성형 AI 기술 변화와 생성형 AI의 선거 영향 및  문제점 분석한다.
### 출처: /Users/user/kykdev/7000_AI/test_langchain/assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf    
""",
    "answer": """
라마 3의 성능은 객관식 문제(MMLU)와 코딩(HumanEval)에서 강점을 보이며, 특히 인간 선호도 측면에서 경쟁 모델을 앞서고 있습니다. 그러나 수학 단어 문제(MATH) 해결이나 대학원생 수준의 객관식 문제(GPQA)에서는 제미나이 프로 1.5에 비해 뒤처집니다. 라마 3은 공개 후 몇 시간 만에 LLM 리더보드에서 1위를 차지하며 역대 가장 빠른 1위 달성을 기록했습니다.

라마 3의 규제 연구 책임자는 국립부경대학교 김주희 교수입니다. 해당 연구는 중앙선거관리위원회의 정책연구용역 과제로서, 차재권(국립부경대학교), 김현정(동아대학교), 조성복(국민대학교) 교수와 박서현, 권수민(이상 국립부경대학교) 연구보조원이 공동으로 참여하였습니다. 이 연구는 생성형 AI의 도입이 선거에 미치는 영향과 문제점을 분석하고 있습니다.    
"""
})

'grounded'